# "Classifiez automatiquement des biens de consommation"
_NLP Cleaning Notebook_

## 0 Preliminaries

### 0.0 Importing Packages and Modules

Checking whether the notebook is on Colab or PC

In [1]:
import sys
is_colab = 'google.colab' in sys.modules
is_colab, sys.executable

(True, '/usr/bin/python3')

Mounting my Drive if on Colab

In [2]:
if is_colab==True:
    from google.colab import files, output, drive
    drive.mount('/gdrive')
    %cd /gdrive
    print("You're on Google Colab")
else:
    print("You're on a PC")

Mounted at /gdrive
/gdrive
You're on Google Colab


Installations and importations required in the virtual environment.

In [3]:
# import os
# if os.getcwd()!='/gdrive/My Drive/--DATA SCIENCE/PROJET6/NOTEBOOKS':
#     os.chdir('/gdrive/My Drive/--DATA SCIENCE/PROJET6/NOTEBOOKS')

In [ ]:
import os
if is_colab==True:
    if os.getcwd()!='/gdrive/My Drive/--DATA SCIENCE/PROJET6/NOTEBOOKS':
        os.chdir('/gdrive/My Drive/--DATA SCIENCE/PROJET6/NOTEBOOKS')
else:
    if not (os.path.exists(os.getcwd()+'/requirements_nlp.txt') \
                     and os.path.exists(os.getcwd()+'/P6_functions.py')):
        print("ERROR: Make sure 'P6_functions.py' and \
'requirements_nlp.txt' are in the current working directory")

!pip install -r requirements_nlp.txt

     |████████████████████████████████| 276kB 9.4MB/s 
     |████████████████████████████████| 143kB 15.8MB/s 
     |████████████████████████████████| 1.1MB 17.1MB/s 
  Created wheel for gTTS-token: filename=gTTS_token-1.1.3-cp36-none-any.whl size=4096 sha256=ecd0f026c75dc8aa4b78544fa9d5c26a81b47dfe71add9742c2b0b2e1226d1e4
  Stored in directory: /root/.cache/pip/wheels/dd/11/61/33f7e51bf545e910552b2255eead2a7cd8ef54064b46dceb34


In [ ]:
from P6_functions import *

Installations (creating the requirements file)

In [ ]:
# !pip install gtts
# !pip install wikipedia2vec==0.2.2
# !pip install category-encoders

In [ ]:
# !pip freeze > requirements_cleaning_nlp.txt

Importation of modules and packages. 

In [ ]:
import io

import string

import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.facecolor']='w'

import nltk
nltk.download('punkt')
nltk.download('stopwords')

# import warnings
# warnings.filterwarnings('ignore')

Setting pandas display options.

In [ ]:
dictPdSettings = {'display.max_rows': 500, 'display.width': 100,
                  'display.max_colwidth': 100,
                  'display.float_format': lambda x: '%.2f' % x}
for k,v in dictPdSettings.items():
  pd.set_option(k,v)

To play audio text-to-speech during execution.

In [ ]:
from IPython.display import Audio
from gtts import gTTS

def speak(text, lang='en'):
    with io.BytesIO() as f:
        gTTS(text=text, lang=lang).write_to_fp(f)
        f.seek(0)
        return Audio(f.read(), autoplay=True)

In [ ]:
speak('Packages and modules successfully imported')

### 0.1 Importing the datasets

Data is composed of 9 distinct .csv files we'll load in a dictionnary of dataframes.

In [ ]:
if is_colab==True:
    # Importing database from my Drive
    print("Try to import data files in the notebook from myDrive...")
else:
    # Importing database from PC
    print("Try to import data files in the notebook from PC ('DATA')...")

df = pd.read_csv("../DATA/flipkart_com-ecommerce_sample_1050.csv",
                 sep=',', 
                 index_col = 'uniq_id',
                 encoding ='utf-8')

print("-----> Importation of .csv in the notebook: OK")

In [ ]:
speak('Datasets successfully imported')

## 1 Data extraction

### 1.1 Categories

Unfolding categories using the 'product_category_tree' colum

In [ ]:
# sample checking
df['product_category_tree'][743]

In [ ]:
# determining the maximum tree depth of categories
ser_depth = df['product_category_tree'].apply(lambda x: x.count('>>'))
max_depth = ser_depth.max()
max_depth

In [ ]:
# Converting the strings in 'product_category_tree' column in 6 categ columns

def str_cleaning(ind, my_str, name_level_cols):
    my_str = my_str.replace("[\"", "").replace("\"]", "")
    tab_str = my_str.split(">>")
    size_tab_str = len(tab_str)
    tup_str = tuple([tab_str[i].strip() if i<size_tab_str else "" \
                     for i in np.arange(max_depth) ])
    return tup_str

name_level_cols = ['cat_level_'+str(i) for i in np.arange(max_depth)]
ser_tuple = df['product_category_tree']\
    .apply(lambda s: str_cleaning(s.index, s, name_level_cols))
df_cat_level = pd.DataFrame([[a,'/'.join([a,b]),'/'.join([a,b,c]),
                              '/'.join([a,b,c,d]),'/'.join([a,b,c,d,e]),
                              '/'.join([a,b,c,d,e,f])] \
                             for a,b,c,d,e,f in ser_tuple.values],
                            columns=name_level_cols, index=df.index)

In [ ]:
# printing number of categories in each level and a sample
display(df_cat_level.nunique(), df_cat_level.sample(3))

Let's see how much items are in each category

In [ ]:
fig = plt.figure(figsize=(25,4))
for i, col in enumerate(df_cat_level.columns,1):
    ax = fig.add_subplot(1,len(df_cat_level.columns), i)
    ser = df_cat_level.groupby(col).size().sort_values(ascending=False)
    ser[0:20].plot.bar(width=0.75, color='grey', ec='k', ax=ax)
    ax.set_title(col+f' ({ser.shape[0]} categories)', fontweight='bold')

The only level that has a balanced set of items is level 0, with 7 categories.
Let's rename these 7 categories:

In [ ]:
df_cat_level['cat_level_0'].unique()

In [ ]:
df_cat_level['category'] = \
    df_cat_level['cat_level_0'].replace({'Home Furnishing': 'Furnishing',
                                        'Baby Care': 'Baby', 
                                        'Watches': 'Watches',
                                        'Home Decor & Festive Needs': 'Decor',
                                        'Kitchen & Dining': 'Kitchen',
                                        'Beauty and Personal Care': 'Beauty',
                                        'Computers': 'Computers'})

### 1.2 Products descriptions

In [ ]:
# extracting only useful data
df_desc_cat = pd.concat([df_cat_level['category'],
                         df[["product_name", "description"]]], axis=1)

In [ ]:
df_desc_cat.head()

In [ ]:
# creation of a corpus of all the descriptions
corpus = ' '.join(df_desc_cat['description'].values)
print("total nb of words in the whole corpus: ", len(corpus.split()))

In [ ]:
df_desc_cat

## 2 Optimisation of the text preprocessing and clustering

In [ ]:
df_res_clust = pd.DataFrame()
df_res_clust['categories'] = df_desc_cat['category']

### FUNCTIONS

#### tokenize_clean

In [ ]:
''' from a sentence, containing words (document):
- tokenizes the words if only composed of alphanumerical data,
- removes stopwords if list is given (stopwords)
- stems the words if stemmer given
NB: This pre-processing function can be used to prepare data for Word2Vec
'''
from nltk.stem.snowball import EnglishStemmer
import spacy

def tokenize_clean(document, stopwords=None, lemmatizer=None, stemmer=None):
    # 1 - tokenizing the words in each description
    tokenizer = nltk.RegexpTokenizer(r'[a-z]+')
    li_words = tokenizer.tokenize(document)
    if stopwords is None: stopwords=[]
    # 2 - lemmatizing or stemming
    if lemmatizer is not None:
        lem_doc = lemmatizer(' '.join(li_words))
        li_words = [token.lemma_ for token in lem_doc]
    elif stemmer is not None:
        li_words = [stemmer.stem(s) for s in li_words]
    # 3 - removing stopwords
    li_words = [s for s in li_words if s not in stopwords]
    # 4 - lower case
    li_words = [s.lower() for s in li_words]
    return li_words

#### compute_doc_terms_df

In [ ]:
''' Takes a pd.Series containing the texts of each description
applies a preprocessing function if given (stopwords, stemming...)
then turn the descriptions in vectors (bow of tf-idf, depending on the avlue of
 tfidf_on)
 returns document term matrix as a dataframe and the list of new excluded words.
'''

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

def compute_doc_terms_df(ser_desc, 
                         preproc_func=None,
                         preproc_func_params=None,
                         vec_params = {'min_df': 1},
                         tfidf_on=False,
                         print_opt=False):

    # ---- Apply a stemming of lemmatization prior to vectorization
    if preproc_func is not None:
        ser_desc = ser_desc.apply(lambda x: preproc_func(x,
                                                         **preproc_func_params))
        ser_desc = ser_desc.apply(lambda x: ' '.join(x))
    else:
        ser_desc = ser_desc
    
    # ---- Vectorization of each of the texts (row)
    if tfidf_on:
        # TF-IDF matrix
        vec = TfidfVectorizer(**vec_params)
    else:
        # BOW matrix (count)
        vec = CountVectorizer(**vec_params)

    doc_term = vec.fit_transform(ser_desc)
    if print_opt:
        print( "Created %d X %d doc_term matrix" % (doc_term.shape[0],
                                                    doc_term.shape[1]))

    # ---- Vocabulary of the document_term matrix
    doc_term_voc = vec.get_feature_names()
    if print_opt:
        print("Vocabulary has %d distinct terms" % len(doc_term_voc))

    # ---- Get the list of the new stop-words
    new_sw = vec.stop_words_
    if print_opt:
        print("Old stop-words list has %d entries" % len(sw) )
        print("New stop-words list has %d entries" % len(new_sw))

    doc_term_df = pd.DataFrame(doc_term.todense(),
                index=ser_desc.index, # each item
                columns=doc_term_voc) # each word

    # document term matrix as a dataframe and the list of new excluded words
    return doc_term_df, new_sw


#### CustNLPTransformer

In [ ]:

''' Builds a customizable column_transformer which parameters can be optimized in a GridSearchCV
CATEGORICAL : three differents startegies for 3 different types of
categorical variables:
- low cardinality: customizable strategy (strat_low_card)
- high cardinality: customizable strategy (strat_high_card)
- boolean or equivalent (2 categories): ordinal
QUANTITATIVE (remainder): 
- StandardScaler

-> EXAMPLE (to use apart from gscv):
cust_enc = CustTransformer(thresh_card=12,
                       strat_binary = 'ord',
                       strat_low_card = 'ohe',
                       strat_high_card = 'loo',
                       strat_quant = 'stand')
cust_enc.fit(X_tr, y1_tr)
cust_enc.transform(X_tr).shape, X_tr.shape

-> EXAMPLE (to fetch names of the modified dataframe):
small_df = df[['Outlier', 'Neighborhood', 'CertifiedPreviousYear',
               'NumberofFloors','ExtsurfVolRatio']]
# small_df.head(2)
cust_trans = CustTransformer()
cust_trans.fit(small_df)
df_enc = cust_trans.transform(small_df)
cust_trans.get_feature_names(small_df)

'''
from sklearn.base import BaseEstimator
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import pandas as pd


class CustNLPTransformer(BaseEstimator):

    def __init__(self, stopwords=None, stemmer=None, lemmatizer=None,
                 min_df=0, max_df=10000, max_features=10000, tfidf_on=False):
        self.stopwords = stopwords
        self.lemmatizer = lemmatizer
        self.stemmer = stemmer
        self.min_df = min_df
        self.max_df = max_df
        self.max_features = max_features
        self.tfidf_on = tfidf_on
        self.preproc_func_params={'stopwords': self.stopwords,
                                  'lemmatizer': self.lemmatizer,
                                  'stemmer': self.stemmer}
        self.vec_params = {'min_df': self.min_df,
                           'max_df': self.max_df,
                           'max_features': self.max_features}

    def __tokenize_clean(self, document, stopwords, lemmatizer, stemmer):
        # 1 - tokenizing the words in each description
        tokenizer = nltk.RegexpTokenizer(r'[a-z]+')
        li_words = tokenizer.tokenize(document)
        if stopwords is None: stopwords=[]
        # 2 - lemmatizing or stemming
        if lemmatizer is not None:
            lem_doc = lemmatizer(' '.join(li_words))
            li_words = [token.lemma_ for token in lem_doc]
        elif stemmer is not None:
            li_words = [stemmer.stem(s) for s in li_words]
        # 3 - removing stopwords
        li_words = [s for s in li_words if s not in stopwords]
        # 4 - lower case
        li_words = [s.lower() for s in li_words]
        return li_words

    # "private" method to be used to apply transformation and get a df
    def __compute_doc_terms_df(self, ser_desc, preproc_func,
                             preproc_func_params,
                            vec_params, tfidf_on):
        # ---- Apply a stemming or lemmatization prior to vectorization
        if preproc_func is not None:
            ser_desc = ser_desc.apply(lambda x: \
                                      preproc_func(x, **preproc_func_params))
            ser_desc = ser_desc.apply(lambda x: ' '.join(x))
        else:
            ser_desc = ser_desc
        # ---- Vectorization of each of the texts (row)
        if tfidf_on:
            # TF-IDF matrix
            vec = TfidfVectorizer(**vec_params)
        else:
            # BOW matrix (count)
            vec = CountVectorizer(**vec_params)
        doc_term = vec.fit_transform(ser_desc)
        # ---- Vocabulary of the document_term matrix
        doc_term_voc = vec.get_feature_names()
        # ---- Get the list of the new stop-words
        new_sw = vec.stop_words_
        doc_term_df = pd.DataFrame(doc_term.todense(),
                                   index=ser_desc.index, # each item
                                   columns=doc_term_voc) # each word
        # document term matrix as a dataframe and the list of new excluded words
        return doc_term_df

    def fit(self, X, y=None):
        # nothing to fit
        return self

    def transform(self, X, y=None):  # to get a dataframe
        df_trans = \
            self.__compute_doc_terms_df(
                ser_desc=X,
                preproc_func=self.__tokenize_clean,
                preproc_func_params=self.preproc_func_params,
                vec_params=self.vec_params,
                tfidf_on=self.tfidf_on
                                   )
        return df_trans

    def fit_transform(self, X, y=None): # self, 
        return self.transform(X, y)

#### GridSearchClust

In [ ]:
'''
Class to optimize clustering score.
Instantiate with a clusterer (estimator), a grid parameter (param_grid)
and a scoring function or a dictionary of functions (scoring)
'''

import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import ParameterGrid
from collections import defaultdict

class GridSearchClust(BaseEstimator, TransformerMixin):

    def __init__(self, estimator, param_grid_estim, param_grid_preproc=None,
                 scoring=None, refit=silhouette_score, greater_is_better=True):

        # Get the parameters
        self.estimator = estimator
        self.param_grid_estim = param_grid_estim
        self.param_grid_preproc = param_grid_preproc
        self.scoring = scoring
        self.refit = refit
        self.greater_is_better = greater_is_better

    def fit(self, X, verbose=False):

        # Initialize the dict of results
        self.results_ = {"scores": {},
                         "params": [],
                         "estimators": [],
                        #  "fit_times": [],
                        #  "nb_clusters": [],
                         "refit_score": []}

        # Iterate upon all combinations of parameters
        estim_score = defaultdict(list)
        for param in ParameterGrid(self.param_grid_estim):

            # Instanciate of the whole estimator with selected parameters
            self.estimator = self.estimator.set_params(**param)

            # Fit the model
            self.estimator.fit(X)

            # If the estimator is a pipe, compute the first steps separately
            if hasattr(self.estimator, 'steps'): # if estimator is a pipeline
                pipe_wo_last_estim = Pipeline(self.estimator.steps[0:-1])
                X_trans = pipe_wo_last_estim.fit_transform(X)

            # Compute the labels
            labels = self.estimator.predict(X)

            # # Measure training time while fitting the model on the data
            # time_train = %timeit -n1 -r1 -o -q self.estimator.fit(X)
            # time_train = time_train.average

            # Compute the refit score
            try:
                if hasattr(self.estimator, 'steps'): # if estimator is a pipeline
                    refit_score = self.refit(X_trans, labels)
                else:
                    refit_score = self.refit(X, labels) # self.scoring['silh'](X, labels)
            except:
                print('calcul du refit score ne marche pas')
                refit_score = np.nan
            
            # Other scores (scoring)
            if not self.scoring:  # if scoring parameter is/are not defined
                estim_score['score'] = {'default_score': self.estimator.score(X)} # default score
            else:  # If scoring parameter is/are defined
                if type(self.scoring) != dict:
                    self.scoring = {'score': self.scoring}
                else:
                    # looping over each score
                    for n_sco, sco in self.scoring.items():
                        try:
                            if hasattr(self.estimator, 'steps'): # if estimator is a pipeline
                                estim_score[n_sco] = estim_score[n_sco] + [sco(X_trans, labels)]
                        except:
                            estim_score[n_sco] = estim_score[n_sco] + [np.nan]
                            print("calcul des scores ne marche pas")
            if verbose: print(estim_score)
            
            # # Computing number of clusters, excluding noise (#-1)
            # nb_clusters = \
            #     len(set(labels)) - (1 if -1 in set(labels) else 0)
            # nb_clusters = int(nb_clusters)
            # if verbose: print(nb_clusters)

            # saving results, parameters and models in a dict
            self.results_["refit_score"].append(refit_score)  # refit score
            # self.results_["scores"].append(estim_score)  # dict of lists of scores
            self.results_["params"].append(param)  # parameters
            self.results_["estimators"].append(self.estimator)  # trained models
            # self.results_["fit_times"].append(time_train)  # training time
            # self.results_["nb_clusters"].append(nb_clusters)  # nb of clusters

        self.results_["scores"] = dict(estim_score)  # dict of lists of scores

        # Selecting best model based on the refit_score
        # -----------------------------------
        # initialisation
        best_estim_index, best_score = None, None  
        # iterating over scores
        for index, score in enumerate(self.results_["refit_score"]):

            # initialisation
            if not best_score:
                best_score = score
                best_estim_index = index

            # if score is better than current best_score
            cond = score > best_score if self.greater_is_better\
                                                 else score < best_score
            if cond:
                    # update the current best_score and current best_estim_index
                    best_score = score
                    best_estim_index = index
        
        # Update attributes of the instance
        self.best_score_ = self.results_["refit_score"][best_estim_index]
        self.best_params_ = self.results_["params"][best_estim_index]
        self.best_estimator_ = self.results_["estimators"][best_estim_index]
        self.best_index_ = best_estim_index
        # self.refit_time_ = self.results_["fit_times"][best_estim_index]

        # refit the best model
        self.best_estimator_.fit(X)
        
        return self

    def predict(self, X):

        # use the .predict method of the estimator on the best model
        return self.best_estim.predict(X)

#### plot_scv_multiscore

In [ ]:
''' Plots the training and test scores obtained during the SearchCV (either Randomized or Grid)
the other parameters are parameters of the best estimator (found by gridsearch)'''


def plot_scv_multi_scores(name_reg, scv, param, title = None, x_log=False, loc='best', figsize = (12, 4)):

    if name_reg is None :
        name_reg = scv.estimator.steps[2][0]

    best_params, df_sel, df_gscv_filt = filters_cv_results(scv,param)
    results = df_gscv_filt

    scoring = scv.scoring
    fig, axs = plt.subplots(1,len(scoring))
    fig.set_size_inches(figsize)
    
    li_colors = ['b', 'r', 'g', 'purple', 'orange', 'pink']
    if len(axs)==1 : axs = [axs]

    # Get the regular np array from the MaskedArray
       
    X_axis = np.array(results['param_'+param], dtype='float')
        
    for scorer, color, ax in zip(sorted(scoring), li_colors[:len(scoring)], axs):
        for sample, style in (('train', '--'), ('test', '-')):
            sample_score_mean = results['mean_%s_%s' % (sample, scorer)].values
            sample_score_std = results['std_%s_%s' % (sample, scorer)].values
            alpha = 0.2 if sample == 'test' else 0.1
            df_ = pd.DataFrame({'param': X_axis,
                                'mean': sample_score_mean,
                                'std': sample_score_std}).sort_values(by='param')
            ax.fill_between(df_['param'],
                            df_['mean'] - df_['std'],
                            df_['mean'] + df_['std'],
                            alpha=alpha, color=color)
            ax.plot(df_['param'], df_['mean'], style, marker='o', markersize=3,
                color=color, alpha=1 if sample == 'test' else 0.7, label=f"{sample}")
            if x_log: ax.set_xscale('log')
            ax.set_title(scorer)
            
        y_min, y_max = ax.get_ylim()
        
        # Plot a dotted vertical line at the best score for that scorer marked by x
        best_index = results['rank_test_%s' % scorer].argmin()
        best_score = results['mean_test_%s' % scorer].iloc[best_index]
        ax.plot([X_axis[best_index], ] * 2, [y_min - abs(y_min)*0.1, best_score],
            linestyle='dotted', color=color, marker='x', markeredgewidth=3, ms=8)
        ax.set_ylim(y_min, y_max)
        ax.set_xlabel(param)
        ax.set_ylabel("Score")
        ax.legend(loc=loc)

        # Annotate the best score for that scorer
        len_str = len("{:.2f}".format(best_score))
        if X_axis[best_index] < np.mean(X_axis):
            x_pos = X_axis[best_index]*(1+0.015*len_str)
        else:
            x_pos = X_axis[best_index]*(1-0.015*len_str)
        y_pos = best_score*1+(y_max-y_min)*0.05
        ax.annotate("{:0.2f}".format(best_score), 
                    (x_pos, y_pos),
                    color = color)  
    if title is not None:
        fig.suptitle(title, fontsize=16, fontweight='bold')
        plt.tight_layout(rect=(0,0,1,0.92))
    else:
        plt.tight_layout()
    plt.show()


#### --- others ---

In [ ]:
from sklearn.cluster import KMeans

def clustering_doc_matrix(doc_matrix_df, name, n_clusters=7):
    # Creating the Kmeans model
    km = KMeans(n_clusters = n_clusters)
    # Fitting the Kmeans model
    km.fit(doc_matrix_df)
    ser = pd.Series(km.labels_,
                    index = doc_matrix_df.index,
                    name = name)
    return ser

### Simple example

In [ ]:
# Stopwords
english_sw = nltk.corpus.stopwords.words('english')
single_let_sw = list(string.ascii_lowercase)
sw = list(set(english_sw + single_let_sw))
len(sw)

In [ ]:
# Stemmer or lemmatizer
from nltk.stem.snowball import EnglishStemmer
import spacy

stemmer = EnglishStemmer()
lemmatizer = spacy.load('en', disable=['parser', 'ner'])

In [ ]:
# Vectorization of the descriptions prior to applying NMF

doc_matrix_df, _ = \
    compute_doc_terms_df(df_desc_cat['description'],
                         preproc_func= tokenize_clean,
                         preproc_func_params = {'stopwords': sw,
                                                'stemmer': None,
                                                'lemmatizer': lemmatizer},
                         vec_params = {'min_df': 5, # min nb of descriptions that must contain the word
                                       'max_features':500}, # max nb of words to keep among the most used
                         tfidf_on=False)

In [ ]:
# Getting the labels of clustering for each set of param (1 param)

max_features_list = [30, 70, 100, 150, 250, 500, 1000]

for max_feat in max_features_list:
    doc_matrix_df, _ = \
        compute_doc_terms_df(df_desc_cat['description'],
                         preproc_func= tokenize_clean,
                         preproc_func_params = {'stopwords': sw,
                                                'stemmer': None,
                                                'lemmatizer': lemmatizer},
                         vec_params = {'min_df': 5, # min nb of descriptions that must contain the word
                                       'max_features':max_feat}, # max nb of words to keep among the most used
                         tfidf_on=False)
    # Appending the best results of the Kmeans clustering
    df_res_clust = pd.concat([df_res_clust,
                            clustering_doc_matrix(doc_matrix_df,
                                                    str(max_feat))], axis=1)

In [ ]:
df_res_clust.sample(5)

In [ ]:
# Comparison of clusters labels with true categories

from sklearn.metrics import adjusted_rand_score

ser_ari_pairs_models = ARI_column_pairs(df_res_clust, first_vs_others=True,
                                        print_opt=True)

In [ ]:
# Plotting the results

fig = plt.figure(figsize=(2,3))
ser_ari_pairs_models.plot.bar(width=0.7, color='grey', ec='k')
plt.ylabel('ARI score')
# plt.title('ARI score comparing the cluster\nlabel prediction of pairs of models')
# plt.gca().set(ylim=(0.85,1))
plt.show()

### GridSearch

#### GridSearch without preprocessing

In [ ]:
from sklearn.metrics import silhouette_score,calinski_harabasz_score, davies_bouldin_score

# Definition of the search space for hyperparameters
param_grid = {"n_clusters": np.arange(3,10)}

# Instanciation of the GridSearch object
gsc = GridSearchClust(estimator=KMeans(random_state=14),
                      param_grid_estim=param_grid,
                      scoring={'silh': silhouette_score,
                               'dav_bould': davies_bouldin_score,
                               'cal_har': calinski_harabasz_score},
                      refit=silhouette_score,
                      greater_is_better=True, # for the refit_score
                      )

# Proceeding the grid search
gsc.fit(doc_matrix_df, verbose=False);

# Displays best parameters
print("Best hyperparameters:", gsc.best_params_)
print("Best Silhouette score:", gsc.best_score_)
gsc_res = gsc.results_
scores_df = pd.DataFrame(gsc_res['scores'],
                         index = pd.DataFrame(gsc_res['params']).iloc[:,0])

#### GridSearch with preprocessing optimization


- créer un custom transformer pour le nlp (init : paramètres) avec .transform
- appliquer un predict dans le Gridsearch qui pourrait s'appliquer à un km ou à un pipe
- vérifier que les fonctions d'affichage (heatmap, courbes) des scores en fonction des paramètres sont compatibles avec ma fonction GridSearchClust (marche toujours sans cv ?)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.metrics import silhouette_score,calinski_harabasz_score, davies_bouldin_score

# Stemmer or lemmatizer
from nltk.stem.snowball import EnglishStemmer
import spacy
stemmer = EnglishStemmer()
lemmatizer = spacy.load('en', disable=['parser', 'ner'])

# clusterer
km = KMeans(random_state=14)

custtrans = CustNLPTransformer(stopwords=sw,
                               stemmer=None,
                               lemmatizer=None,
                               min_df=1,
                               max_df=200, # nb max de documents dans lequel le mot apparaît (150 documents par catégorie)
                               max_features=500,
                               tfidf_on=False
                               )

# Defining the pipeline to be executed and optimized by the GridSearch
pipe = Pipeline([('custtrans', custtrans),
                 ('clusterer', km)])

# Defining the list of params to be tested in the GridSearchClust
param_grid = {'custtrans__stopwords': [None], # [sw, None],
              'custtrans__stemmer': [None],#[stemmer, None],
              'custtrans__lemmatizer': [None],
              'custtrans__min_df': [1],
              'custtrans__max_df': [200], # nb max de documents dans lequel le mot apparaît (150 documents par catégorie)
              'custtrans__max_features': [300,500,1000],
              'custtrans__tfidf_on': [True], #[True, False],
              'clusterer__n_clusters': np.arange(3,6)
              }

# Instanciation of the GridSearch object
gsc = GridSearchClust(estimator=pipe,
                      param_grid_estim=param_grid,
                      scoring={'silh': silhouette_score,
                               'dav_bould': davies_bouldin_score,
                               'cal_har': calinski_harabasz_score},
                      refit=silhouette_score,
                      greater_is_better=True) # for the refit_score

# Proceeding the grid search
gsc.fit(df_desc_cat['description'], verbose=False);

# Displays best parameters
print("Best hyperparameters:", gsc.best_params_)
print("Best Silhouette score:", gsc.best_score_)
gsc_res = gsc.results_
scores_df = pd.DataFrame(gsc_res['scores'],
                         index = pd.DataFrame(gsc_res['params']).iloc[:,0])


In [ ]:
gsc_res.keys()

In [ ]:
gsc_res['scores']

#### Plotting the results of the clustering optimization

In [ ]:
dict_scv_params, models_file_name, l_curves_file_name, perm_imp_file_name = \
        set_dict_scv_params(X_tr, y_tr, target, log_on=False, refit=refit_score)

df_res = run_optimization(name_reg, reg, param_grid, models_file_name, dict_models,
                          pipe, dict_scv_params, skf, df_res, search_strat, n_iter)

In [ ]:
# Best parameters
dict_models[name_reg].best_params_

In [ ]:
# Cross-validation scores on the training set
df_res[name_reg].to_frame().T.dropna(1)

In [ ]:
# Multiscoring with hyperparameter tuning
scv = dict_models[name_reg]
plot_scv_multi_scores(name_reg, scv, param = name_reg+'__n_neighbors',
                      title = None, figsize = (15, 3))

In [ ]:
# Effect of encoding on performance

param1 = name_reg+'__'+'n_neighbors'
param2 = name_reg+'__'+'p'
param3 = 'preproc__cust_trans__strat_low_card'
param4 = 'preproc__cust_trans__strat_high_card'
param5 = 'preproc__cust_trans__strat_quant'
score = 'r2'

fig = plt.figure(figsize=(20,3))
ax = fig.add_subplot(1,4,1)
plot_2D_hyperparam_opt(scv=dict_models[name_reg], params=[param1, param2],
                       score = score, ax=ax)
ax = fig.add_subplot(1,4,2)
plot_2D_hyperparam_opt(scv=dict_models[name_reg], params=[param1, param3],
                       score = score, ax=ax)
ax = fig.add_subplot(1,4,3)
plot_2D_hyperparam_opt(scv=dict_models[name_reg], params=[param1, param4],
                       score = score, ax=ax)
ax = fig.add_subplot(1,4,4)
plot_2D_hyperparam_opt(scv=dict_models[name_reg], params=[param1, param5],
                       score = score, ax=ax)

In [ ]:
from sklearn.metrics import silhouette_score,calinski_harabasz_score, davies_bouldin_score

# Definition of the search space for hyperparameters
param_grid = {"n_clusters": np.arange(3,10)}

# Instanciation of the GridSearch object
gsc = GridSearchClust(estimator=KMeans(random_state=14),
                      param_grid_estim=param_grid,
                      scoring={'silh': silhouette_score,
                               'dav_bould': davies_bouldin_score,
                               'cal_har': calinski_harabasz_score},
                      refit=silhouette_score,
                      greater_is_better=True, # for the refit_score
                      )

# Proceeding the grid search
gsc.fit(doc_matrix_df, verbose=False);

# Displays best parameters
print("Best hyperparameters:", gsc.best_params_)
print("Best Silhouette score:", gsc.best_score_)
gsc_res = gsc.results_
scores_df = pd.DataFrame(gsc_res['scores'],
                         index = pd.DataFrame(gsc_res['params']).iloc[:,0])

In [ ]:
scores_df